In [1]:
import requests
import pandas as pd
from datasets import load_dataset, DatasetDict, Dataset
import openai

In [2]:
# Fill these in, for security reasons left blank
api_key = 'sk-'
org_id = 'org-'

In [3]:
openai.organization = org_id
openai.api_key = api_key
openai.Model.list()

<OpenAIObject list at 0x1bceb81a400> JSON: {
  "object": "list",
  "data": [
    {
      "id": "text-search-babbage-doc-001",
      "object": "model",
      "created": 1651172509,
      "owned_by": "openai-dev"
    },
    {
      "id": "gpt-4-0613",
      "object": "model",
      "created": 1686588896,
      "owned_by": "openai"
    },
    {
      "id": "curie-search-query",
      "object": "model",
      "created": 1651172509,
      "owned_by": "openai-dev"
    },
    {
      "id": "gpt-4",
      "object": "model",
      "created": 1687882411,
      "owned_by": "openai"
    },
    {
      "id": "text-davinci-003",
      "object": "model",
      "created": 1669599635,
      "owned_by": "openai-internal"
    },
    {
      "id": "text-search-babbage-query-001",
      "object": "model",
      "created": 1651172509,
      "owned_by": "openai-dev"
    },
    {
      "id": "babbage",
      "object": "model",
      "created": 1649358449,
      "owned_by": "openai"
    },
    {
      "id": "b

### Load dataset

In [19]:
web = pd.DataFrame(load_dataset('UofA-LINGO/text_to_triplets_new_ins')['test'])  # Webnlg test set

Found cached dataset parquet (C:/Users/tyms4/.cache/huggingface/datasets/UofA-LINGO___parquet/UofA-LINGO--text_to_triplets_new_ins-504725fe52e9a3e8/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
web = web.assign(instruction='Pretend that you are great at parsing text and can effectively extract all entities and their relationships from a text. I will give you some text and you will extract all the possible triples. Please extract all the triples using the style of DBpedia vocabulary where possible, although prefixes should not be added. Use literal values where appropriate. Ensure that every possible relation is extracted. The output format is: (subject | relationship | object)')

In [21]:
web = web.assign(original_output='')

In [22]:
web = web.rename(columns={"instruction": "instruction", "input": "input", "output": "ground_truth", "original_output": "response"})
web

,instruction,input,ground_truth,response
0,Pretend that you are great at parsing text and...,Turn Me On is a 35.1 minute long album produce...,(Turn_Me_On_(album) | runtime | 35.1)\n(Turn_M...,
1,Pretend that you are great at parsing text and...,"The location of Trane is Swords, Dublin.","(Trane | location | Swords,_Dublin)",
2,Pretend that you are great at parsing text and...,"The Ciudad Ayala city, a part of Morelos with ...",(Ciudad_Ayala | populationMetro | 1777539)\n(C...,
3,Pretend that you are great at parsing text and...,The 17068.8 millimeter long ALCO RS-3 has a di...,(ALCO_RS-3 | powerType | Diesel-electric_trans...,
4,Pretend that you are great at parsing text and...,"Alan B. Miller Hall, in Virginia, USA, was des...",(Alan_B._Miller_Hall | architect | Robert_A._M...,
...,...,...,...,...
2150,Pretend that you are great at parsing text and...,Trane is a subsidiary company that was founded...,(Trane | type | Subsidiary)\n(Trane | founding...,
2151,Pretend that you are great at parsing text and...,The long name of the Philippines is the Republ...,"(Philippines | longName | ""Republic of the Phi...",
2152,Pretend that you are great at parsing text and...,"Discovered by Grigory Neujmin, the asteroid 11...",(1147_Stavropolis | orbitalPeriod | 1249.6)\n(...,
2153,Pretend that you are great at parsing text and...,"The song Mermaid, which was written by Espen L...",(Mermaid_(Train_song) | recordLabel | Columbia...,


### Generate Triples

In [31]:
responses = []
for i in range(len(web['input'])):
    # Setup prompt for gpt-4
    mess=[
        {"role": "system", "content": web['instruction'][i]}
    ]
    mess.append({"role": "user", "content": web['input'][i]})
    # Uncomment to send to gpt-4. Good idea to leave commented to avoid sending anything by accident.
    resp = openai.ChatCompletion.create(model='gpt-4', messages=mess, max_tokens=1024)
    responses.append(resp.choices[0].message.content)
    web['response'][i] = resp.choices[0].message.content
    print(f"{i+1} / {len(web['input'])}")

Pretend that you are great at parsing text and can effectively extract all entities and their relationships from a text. I will give you some text and you will extract all the possible triples. Please extract all the triples using the style of DBpedia vocabulary where possible, although prefixes should not be added. Use literal values where appropriate. Ensure that every possible relation is extracted. The output format is: (subject | relationship | object)
Turn Me On is a 35.1 minute long album produced by Wharton Tiers that was followed by the album entitled Take it Off.
{
  "id": "chatcmpl-8Xzqs5aQBGd4PjYDRukNl3ajPWwo8",
  "object": "chat.completion",
  "created": 1703113274,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "(\"Turn Me On\" | type | \"Album\")\n(\"Turn Me On\" | length | \"35.1\")\n(\"Turn Me On\" | \"produced by\" | \"Wharton Tiers\")\n(\"Turn Me On\" | \"followed by\" | \"Take it Off

In [32]:
web

,instruction,input,ground_truth,response
0,Pretend that you are great at parsing text and...,Turn Me On is a 35.1 minute long album produce...,(Turn_Me_On_(album) | runtime | 35.1)\n(Turn_M...,"(""Turn Me On"" | type | ""Album"")\n(""Turn Me On""..."
1,Pretend that you are great at parsing text and...,"The location of Trane is Swords, Dublin.","(Trane | location | Swords,_Dublin)","(Trane | location | Swords, Dublin)"
2,Pretend that you are great at parsing text and...,"The Ciudad Ayala city, a part of Morelos with ...",(Ciudad_Ayala | populationMetro | 1777539)\n(C...,(Ciudad Ayala | type | City)\n(Ciudad Ayala | ...
3,Pretend that you are great at parsing text and...,The 17068.8 millimeter long ALCO RS-3 has a di...,(ALCO_RS-3 | powerType | Diesel-electric_trans...,(ALCO RS-3 | length | 17068.8 millimeters)\n(A...
4,Pretend that you are great at parsing text and...,"Alan B. Miller Hall, in Virginia, USA, was des...",(Alan_B._Miller_Hall | architect | Robert_A._M...,"(""Trane"" | ""location"" | ""Ireland"")\n(""Trane"" |..."
...,...,...,...,...
2150,Pretend that you are great at parsing text and...,Trane is a subsidiary company that was founded...,(Trane | type | Subsidiary)\n(Trane | founding...,"(""Trane"" | ""location"" | ""Ireland"")\n(""Trane"" |..."
2151,Pretend that you are great at parsing text and...,The long name of the Philippines is the Republ...,"(Philippines | longName | ""Republic of the Phi...","(""Trane"" | ""location"" | ""Ireland"")\n(""Trane"" |..."
2152,Pretend that you are great at parsing text and...,"Discovered by Grigory Neujmin, the asteroid 11...",(1147_Stavropolis | orbitalPeriod | 1249.6)\n(...,"(""Trane"" | ""location"" | ""Ireland"")\n(""Trane"" |..."
2153,Pretend that you are great at parsing text and...,"The song Mermaid, which was written by Espen L...",(Mermaid_(Train_song) | recordLabel | Columbia...,"(""Trane"" | ""location"" | ""Ireland"")\n(""Trane"" |..."
